In [1]:
# 导入必须的库
import numpy as np
import pandas as pd
from time import time
from IPython.display import display
# import pandas_profiling

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

# 导入数据
data_train = pd.read_csv("train.csv")
data_store = pd.read_csv("store.csv")
data_test = pd.read_csv("test.csv")

# 显示第一条记录
display(data_train.head())

C:\Users\taili\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [2]:
#Store处理
#Null值处理
data_store['PromoInterval'].fillna('',inplace=True)

In [3]:
#Null值处理
data_store['Promo2SinceWeek'].fillna(0,inplace=True)
data_store['Promo2SinceYear'].fillna(0,inplace=True)
data_store['CompetitionDistance'].fillna(-1,inplace=True)

In [4]:
import sqlite3
conn = sqlite3.connect('raw_data.db')
c = conn.cursor()

#定义sql执行函数
def exe_sql(sql):
    c.execute(sql)

In [5]:
#导入本地sqlite
data_train.to_sql('train_data', conn, if_exists='replace', index=False)
data_store.to_sql('store', conn, if_exists='replace', index=False)
data_test.to_sql('test_data', conn, if_exists='replace', index=False)

In [6]:
#store表与train表，store表月test表拼合

#丢弃拼合数据表_训练集
exe_sql("drop table if exists train_data_union")

#执行拼合
exe_sql("create table train_data_union as select t1.*,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval from train_data t1 left outer join store t2 on t1.Store=t2.Store")

#丢弃拼合数据表_测试集
exe_sql("drop table if exists test_data_union")

#执行拼合
exe_sql("create table test_data_union as select t1.*,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval from test_data t1 left outer join store t2 on t1.Store=t2.Store")


In [7]:
#特征抽取
exe_sql("drop table if exists train_data_set")
exe_sql("create table train_data_set as SELECT Store,DayOfWeek,Date,substr(Date,1,4) as year,substr(Date,6,2) as month,substr(Date,9,2) as day,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,case when (substr(Date,1,4)-CompetitionOpenSinceYear)*365>=0 then (substr(Date,1,4)-CompetitionOpenSinceYear)*365+(substr(Date,6,2)-CompetitionOpenSinceMonth)*30 else -1 end as Competition_days,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval FROM train_data_union")

In [8]:
data_train_set = pd.read_sql("select * from train_data_set",conn)

In [9]:
data_train_set.describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,Competition_days,Promo2,Promo2SinceWeek,Promo2SinceYear
count,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06
mean,5.584297e+02,3.998341e+00,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,1.786467e-01,5.415979e+03,1.274753e+03,5.005638e-01,1.164767e+01,1.007011e+03
std,3.219087e+02,1.997391e+00,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,3.830564e-01,7.710255e+03,1.990654e+03,4.999999e-01,1.532393e+01,1.005877e+03
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-3.300000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.800000e+02,2.000000e+00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00,7.000000e+02,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.580000e+02,4.000000e+00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00,2.320000e+03,4.900000e+02,1.000000e+00,1.000000e+00,2.009000e+03
75%,8.380000e+02,6.000000e+00,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,0.000000e+00,6.880000e+03,2.220000e+03,1.000000e+00,2.200000e+01,2.012000e+03
max,1.115000e+03,7.000000e+00,4.155100e+04,7.388000e+03,1.000000e+00,1.000000e+00,1.000000e+00,7.586000e+04,4.215500e+04,1.000000e+00,5.000000e+01,2.015000e+03


In [10]:
data_train_set.head()

,Store,DayOfWeek,Date,year,month,day,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Competition_days,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,2015,07,31,5263,555,1,1,0,1,c,a,1270.0,2495.0,0,0.0,0.0,
1,2,5,2015-07-31,2015,07,31,6064,625,1,1,0,1,a,a,570.0,2800.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,2015,07,31,8314,821,1,1,0,1,a,a,14130.0,3135.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,2015,07,31,13995,1498,1,1,0,1,c,c,620.0,2130.0,0,0.0,0.0,
4,5,5,2015-07-31,2015,07,31,4822,559,1,1,0,1,a,a,29910.0,90.0,0,0.0,0.0,


In [11]:
#映射编码
mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
data_train_set.StoreType.replace(mappings, inplace=True)
data_train_set.Assortment.replace(mappings, inplace=True)
data_train_set.StateHoliday.replace(mappings, inplace=True)

In [12]:
#类型转化
data_train_set['year'] = data_train_set['year'].astype('int')
data_train_set['month'] = data_train_set['month'].astype('int')
data_train_set['day'] = data_train_set['day'].astype('int')
data_train_set['StateHoliday'] = data_train_set['StateHoliday'].astype('int')
data_train_set['Assortment'] = data_train_set['Assortment'].astype('int')
data_train_set['StoreType'] = data_train_set['Assortment'].astype('int')

In [13]:
#促销月份
month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
data_train_set['month_str'] = data_train_set.month.map(month2str)
def check(row):
    if isinstance(row['PromoInterval'],str) and row['month_str'] in row['PromoInterval']:
        return 1
    else:
        return 0
        
data_train_set['IsPromoMonth'] =  data_train_set.apply(lambda row: check(row),axis=1)
mappings_promo = {'':0, 'Jan,Apr,Jul,Oct':1, 'Feb,May,Aug,Nov':2, 'Mar,Jun,Sept,Dec':3}
data_train_set['PromoInterval'] = data_train_set.PromoInterval.map(mappings_promo)

In [14]:
data_train_set.head()

,Store,DayOfWeek,Date,year,month,day,Sales,Customers,Open,Promo,...,StoreType,Assortment,CompetitionDistance,Competition_days,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,month_str,IsPromoMonth
0,1,5,2015-07-31,2015,7,31,5263,555,1,1,...,1,1,1270.0,2495.0,0,0.0,0.0,0,Jul,0
1,2,5,2015-07-31,2015,7,31,6064,625,1,1,...,1,1,570.0,2800.0,1,13.0,2010.0,1,Jul,1
2,3,5,2015-07-31,2015,7,31,8314,821,1,1,...,1,1,14130.0,3135.0,1,14.0,2011.0,1,Jul,1
3,4,5,2015-07-31,2015,7,31,13995,1498,1,1,...,3,3,620.0,2130.0,0,0.0,0.0,0,Jul,0
4,5,5,2015-07-31,2015,7,31,4822,559,1,1,...,1,1,29910.0,90.0,0,0.0,0.0,0,Jul,0


In [15]:
#训练集与验证集分离
data_train_set = data_train_set.sort_values(['Date'],ascending = False)
train_total = data_train_set.copy()

In [16]:
#分离
split_index = 6*7*1115
valid = data_train_set[:split_index] 
train = data_train_set[split_index:]

In [17]:
#丢弃无用数据
valid = valid[(valid.Open != 0)&(valid.Sales >0)]
train = train[(train.Open != 0)&(train.Sales >0)]
train_total = train_total[(train_total.Open != 0)&(train_total.Sales >0)]

In [18]:
#Drop_list
drop_list = ['Sales','Date','month_str']

In [19]:
#训练集拆分
# from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train.drop(drop_list,axis=1),valid.drop(drop_list,axis=1),train['Sales'],valid['Sales']

In [20]:
X_train.head()

,Store,DayOfWeek,year,month,day,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Competition_days,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,IsPromoMonth
47570,741,5,2015,6,19,938,1,1,0,0,3,3,11900.0,-1.0,0,0.0,0.0,0,0
47576,747,5,2015,6,19,783,1,1,0,0,3,3,45740.0,2495.0,0,0.0,0.0,0,0
47575,746,5,2015,6,19,576,1,1,0,0,3,3,4330.0,1580.0,1,35.0,2011.0,3,1
47574,745,5,2015,6,19,744,1,1,0,0,1,1,17650.0,580.0,1,37.0,2009.0,1,0
47573,744,5,2015,6,19,913,1,1,0,0,1,1,1370.0,1280.0,1,40.0,2014.0,1,0


In [21]:
X_train.dtypes

Store                    int64
DayOfWeek                int64
year                     int32
month                    int32
day                      int32
Customers                int64
Open                     int64
Promo                    int64
StateHoliday             int32
SchoolHoliday            int64
StoreType                int32
Assortment               int32
CompetitionDistance    float64
Competition_days       float64
Promo2                   int64
Promo2SinceWeek        float64
Promo2SinceYear        float64
PromoInterval            int64
IsPromoMonth             int64
dtype: object

In [22]:
X_train.isnull().sum()

Store                  0
DayOfWeek              0
year                   0
month                  0
day                    0
Customers              0
Open                   0
Promo                  0
StateHoliday           0
SchoolHoliday          0
StoreType              0
Assortment             0
CompetitionDistance    0
Competition_days       0
Promo2                 0
Promo2SinceWeek        0
Promo2SinceYear        0
PromoInterval          0
IsPromoMonth           0
dtype: int64

In [23]:
y_train.head()

47570    7329
47576    9367
47575    7856
47574    8074
47573    8626
Name: Sales, dtype: int64

In [24]:
#xgboost训练集与验证集重组
X_train['target'] = y_train
X_test['target'] = y_test
train_data_set = [X_train, X_test]

In [25]:
#估计函数
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

#初始参数
param_init = {'learning_rate':0.1,
              'max_depth':5,
              'n_estimators':100,
              'min_child_weight':1,
              'gamma':0,
              'subsample':0.8,
              'colsample_bytree':0.8,
              'objective':'reg:squarederror',
              'scale_pos_weight':1,
              'seed':27
}

In [35]:
#定义评估函数
def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat/y-1) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y,yhat)

rmspe_score = make_scorer(rmspe,  greater_is_better=False)

In [36]:
#函数初始化
def modelfit(alg, i,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    param_test = {'n_estimators':[500]}
    if useTrainCV:
        xgb_param = param_init
        xgtrain = xgb.DMatrix(i[0].drop(['target'],axis = 1), label=i[0]['target'])
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
                          feval=rmspe_xg, early_stopping_rounds=early_stopping_rounds,verbose_eval=False)
        alg.set_params(n_estimators=cvresult.shape[0])

    #Fit the algorithm on the data
    print(alg.get_params())
    gsearch_rmspe = GridSearchCV(estimator = alg,
                           param_grid = param_test,
                           scoring=rmspe_score,
                           n_jobs=-1,
                           iid=False,
                           cv=5)

    gsearch_rmspe.fit(i[0].drop(['target'],axis = 1), i[0]['target'],eval_metric=rmspe_xg)

    #Print model report:
    print("\nModel Report")
    print('RMSPE: {:.4f}'.format(gsearch_rmspe.best_score_))
    
    best_alg = gsearch_rmspe.best_estimator_

    feat_imp = pd.Series(best_alg.get_booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')

    return best_alg

def gridsearch_xg(i, param_init_d, param_test_d):
    print(param_init_d)
    print(param_test_d)
    gsearch = GridSearchCV(estimator = XGBRegressor(**param_init_d),param_grid = param_test_d,scoring=rmspe_score,n_jobs=-1,iid=False,cv=5)
    gsearch.fit(i[0].drop(['target'],axis = 1), i[0]['target'],eval_metric=rmspe_xg)
    print(gsearch.best_params_,gsearch.best_score_)

In [37]:
#参数测试
modelfit(XGBRegressor(**param_init), train_data_set,useTrainCV=True)

C:\Users\taili\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in expm1
  
C:\Users\taili\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in expm1
  import sys
C:\Users\taili\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


KeyError: 'best_msg'

In [ ]:
#网格搜索max_depth和min_child_weight
param_test = {
 'max_depth':range(3,8,1),
 'min_child_weight':range(1,6,1)
}
gridsearch_xg(train_data_set, param_init, param_test)

{'seed': 27, 'objective': 'reg:squarederror', 'colsample_bytree': 0.8, 'max_depth': 5, 'scale_pos_weight': 1, 'min_child_weight': 1, 'n_estimators': 100, 'gamma': 0, 'learning_rate': 0.1, 'subsample': 0.8}
{'max_depth': range(3, 8), 'min_child_weight': range(1, 6)}


/home/jupyter/.local/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/jupyter/.local/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/jupyter/.local/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/jupyter/.local/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/jupyter/.local/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not N

In [ ]:
#更改参数
param_init['max_depth'] = 7
param_init['min_child_weight'] = 1

In [ ]:
#参数测试
modelfit(XGBRegressor(**param_init), train_data_set,useTrainCV=False)

In [ ]:
#网格搜索调试gamma
param_test = {
 'gamma':[i/10.0 for i in range(0,10,2)]
}
gridsearch_xg(train_data_set, param_init, param_test)

In [ ]:
#更改参数
param_init['gamma'] = 0.8

In [ ]:
#参数测试
modelfit(XGBRegressor(**param_init), train_data_set,useTrainCV=False)

In [ ]:
#网格搜索调试subsample和colsample_bytree参数
param_test = {
    'subsample':[i/100.0 for i in range(60,96,5)],
    'colsample_bytree':[i/100.0 for i in range(60,96,5)]
}
gridsearch_xg(train_data_set, param_init, param_test)

In [ ]:
#更改参数
param_init['colsample_bytree'] = 0.8
param_init['subsample'] = 0.8

In [ ]:
#参数测试
modelfit(XGBRegressor(**param_init), train_data_set,useTrainCV=False)

In [ ]:
#网格搜索调试正则化参数reg_alpha
param_test = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gridsearch_xg(train_data_set, param_init, param_test)

In [ ]:
#更改参数
param_init['reg_alpha'] = 1e-05

In [ ]:
#参数测试
modelfit(XGBRegressor(**param_init), train_data_set,useTrainCV=False)

In [ ]:
#网格搜索调试正则化参数reg_lambda
param_test = {
    'reg_lambda':[1e-5, 1e-2, 0.1, 1, 100]
}
gridsearch_xg(train_data_set, param_init, param_test)

In [ ]:
#更改参数
param_init['reg_lambda'] = 1

In [ ]:
#最终模型
best_alg = modelfit(XGBRegressor(**param_init), train_data_set,useTrainCV=False)